# COVID-19 mRNA Vaccine Degradation Prediction


**Context: The most promising COVID-19 vaccine candidates use messenger RNA molecules (mRNA)** to help the patient develop immunity. Unfortunately, mRNA molecules are extremely fragile, and in certain circumstances spontaneously degrade. Current best scientific understanding lacks a means to analyze mRNA molecule candidates for areas of likely degradation.

**Goal: Predict degradation rates for each part of an RNA molecule.**


In [11]:
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from time import sleep

from tensorflow.keras.preprocessing.text import Tokenizer

## Automated hyper-parameter optimization

In [2]:
# Hyper-parameters
LR = 1e-3
EPOCHS = 1
BATCH_SIZE = 64
EMBED_DIM = 100
HIDDEN_DIM = 128
DROPOUT = .5
SP_DROPOUT = .3
TRAIN_SEQUENCE_LENGTH = 107

## Load data

In [ ]:
train_df = pd.read_json("data/train.json", lines=True)
test_df = pd.read_json("data/test.json", lines=True)

In [ ]:
sample_submission_df = pd.read_csv("data/sample_submission.csv")

## Preprocess data

In [4]:
symbols = "().ACGUBEHIMSX"
feat_cols = ["sequence", "structure", "predicted_loop_type"]
target_cols = ["reactivity", "deg_Mg_pH10", "deg_Mg_50C", "deg_pH10", "deg_50C"]
error_cols = ["reactivity_error", "deg_error_Mg_pH10", "deg_error_Mg_50C", "deg_error_pH10", "deg_error_50C"]

In [5]:
tokenizer = Tokenizer(char_level=True, filters="")
tokenizer.fit_on_texts(symbols)

In [6]:
# get the number of elements in the vocabulary
vocab_size = len(tokenizer.word_index) + 1

In [7]:
def process_features(example):
    sequence_sentences = example[0]
    structure_sentences = example[1]
    loop_sentences = example[2]
    
    # transform character sequences into number sequences
    sequence_tokens = np.array(
        tokenizer.texts_to_sequences(sequence_sentences)
    )
    structure_tokens = np.array(
        tokenizer.texts_to_sequences(structure_sentences)
    )
    loop_tokens = np.array(
        tokenizer.texts_to_sequences(loop_sentences)
    )
    
    # concatenate the tokenized sequences
    sequences = np.stack(
        (sequence_tokens, structure_tokens, loop_tokens),
        axis=1
    )
    sequences = np.transpose(sequences, (2, 0, 1))
    
    prepared = sequences.tolist()
    
    return prepared[0]

In [ ]:
def process_labels(df):
    df = df.copy()
    
    labels = np.array(df[target_cols].values.tolist())
    labels = np.transpose(labels, (0, 2, 1))
    
    return labels

In [ ]:
public_test_df = test_df.query("seq_length == 107")
private_test_df = test_df.query("seq_length == 130")

In [ ]:
x_train = [process_features(row.tolist()) for _, row in train_df[feat_cols].iterrows()]
y_train = process_labels(train_df)

unprocessed_x_public_test = [row.tolist() for _, row in public_test_df[feat_cols].iterrows()]
unprocessed_x_private_test = [row.tolist() for _, row in private_test_df[feat_cols].iterrows()]

# Define and train the model

In [ ]:
def gru_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
         tf.keras.layers.GRU(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer = 'orthogonal')
    )

In [ ]:
def lstm_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(hidden_dim, dropout=dropout, return_sequences=True, kernel_initializer = 'orthogonal')
    )

In [ ]:
def build_model(vocab_size, seq_length=TRAIN_SEQUENCE_LENGTH, pred_len=68,
                embed_dim=EMBED_DIM,
                hidden_dim=HIDDEN_DIM, dropout=DROPOUT, sp_dropout=SP_DROPOUT):
    inputs = tf.keras.layers.Input(shape=(seq_length, 3))

    embed = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)
    
    reshaped = tf.reshape(
        embed, shape=(-1, embed.shape[1],  embed.shape[2] * embed.shape[3])
    )
    
    hidden = tf.keras.layers.SpatialDropout1D(sp_dropout)(reshaped)
    
    hidden = gru_layer(hidden_dim, dropout)(hidden)
    hidden = lstm_layer(hidden_dim, dropout)(hidden)
    
    truncated = hidden[:, :pred_len]
    
    out = tf.keras.layers.Dense(5, activation="linear")(truncated)
    
    model = tf.keras.Model(inputs=inputs, outputs=out)
    
    return model

In [ ]:
model = build_model(vocab_size)

In [ ]:
class MeanColumnwiseRMSE(tf.keras.losses.Loss):
    def __init__(self, name='MeanColumnwiseRMSE'):
        super().__init__(name=name)

    def call(self, y_true, y_pred):
        colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
        return tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)

In [ ]:
model.compile(tf.optimizers.Adam(learning_rate=LR), loss=MeanColumnwiseRMSE())

In [ ]:
history = model.fit(np.array(x_train), np.array(y_train), 
                    validation_split=.1, batch_size=BATCH_SIZE, epochs=EPOCHS)

In [ ]:
validation_loss = history.history.get("val_loss")[0]

## Evaluate the model

In [ ]:
model_public = build_model(vocab_size, seq_length=107, pred_len=107)
model_private = build_model(vocab_size, seq_length=130, pred_len=130)

model_public.set_weights(model.get_weights())
model_private.set_weights(model.get_weights())

In [ ]:
public_preds = model_public.predict(np.array([process_features(x) for x in unprocessed_x_public_test]))
private_preds = model_private.predict(np.array([process_features(x) for x in unprocessed_x_private_test]))

In [ ]:
print(validation_loss)

In [3]:
from kale.common.serveutils import serve


In [8]:
kfserver = serve(model, preprocessing_fn=process_features, preprocessing_assets={'tokenizer': tokenizer})


2021-02-22 17:54:16 Kale serveutils:217       [INFO]     Starting serve procedure for model '<tensorflow.python.keras.engine.functional.Functional object at 0x7f8ec48fe668>'
2021-02-22 17:54:16 Kale jputils:324          [INFO]     Retrieving absolute path of the active notebook
2021-02-22 17:54:16 Kale marshalling          [INFO]     Saving function object using Function backend: transformer_function
2021-02-22 17:54:16 Kale marshalling          [WARNING]  No backends found for type keras_preprocessing.text.Tokenizer (object). Falling back to default backend.
2021-02-22 17:54:16 Kale marshalling          [INFO]     Saving generic object using Default backend: tokenizer
2021-02-22 17:54:16 Kale podutils:82          [INFO]     Getting the current container name...
2021-02-22 17:54:16 Kale podutils:88          [INFO]     Using NB_PREFIX env var '/notebook/kubeflow-user/vaccine-serve'. Container name: 'vaccine-serve'
2021-02-22 17:54:16 Kale serveutils:242       [INFO]     Model is contain

[WARNING]:tensorflow:From /home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
[WARNING]:tensorflow:From /home/jovyan/.local/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py:111: Layer.updates (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
[INFO]:tensorflow:Assets written to: /home/jovyan/.kale.kfserving.model.dir/model.tfkeras/1/assets


2021-02-22 17:54:41 Kale serveutils:247       [INFO]     Model saved successfully at '/home/jovyan/.kale.kfserving.model.dir/model.tfkeras'
2021-02-22 17:54:42 Kale rokutils:60          [INFO]     Taking a snapshot of PVC workspace-vaccine-serve-w64b9p32q in namespace kubeflow-user ...
2021-02-22 17:54:42 Kale rokutils:259         [INFO]     Creating Rok bucket 'serving'...
2021-02-22 17:54:42 Kale rokutils:276         [INFO]     Successfully created Rok bucket 'serving'
2021-02-22 17:54:49 Kale rokutils:79          [INFO]     Successfully took Rok snapshot
2021-02-22 17:54:49 Kale rokutils:219         [INFO]     Creating new PVC 'vaccine-serve-0-nffe8-pvc-78yjw' from Rok version 0cd1eb91-bd1a-42ae-80ac-60ff0a260092 ...
2021-02-22 17:54:49 Kale rokutils:231         [INFO]     Using Rok url: http://rok.rok.svc.cluster.local/swift/v1/kubeflow-user/serving/workspace-vaccine-serve-w64b9p32q?version=0cd1eb91-bd1a-42ae-80ac-60ff0a260092
2021-02-22 17:54:49 Kale rokutils:253         [INFO]   

In [9]:
data = json.dumps({"instances": unprocessed_x_public_test})


In [ ]:
while True:
    kfserver.predict(data)
    sleep(5)

2021-02-22 17:58:31 Kale serveutils:152       [INFO]     Sending a request to the InferenceService...
2021-02-22 17:58:31 Kale serveutils:153       [INFO]     Getting InferenceService's host...
2021-02-22 17:58:31 Kale serveutils:156       [INFO]     Sending request to InferenceService...
2021-02-22 17:58:36 Kale serveutils:162       [INFO]     Response: {"predictions": [[[0.923311, 1.1981051, 1.11143434, 2.45272899, 1.15551567] ..... 27186], [0.327044964, 0.49445653, 0.401043564, 0.441165388, 0.385348141]]]}
2021-02-22 17:58:41 Kale serveutils:152       [INFO]     Sending a request to the InferenceService...
2021-02-22 17:58:41 Kale serveutils:153       [INFO]     Getting InferenceService's host...
2021-02-22 17:58:41 Kale serveutils:156       [INFO]     Sending request to InferenceService...
2021-02-22 17:58:46 Kale serveutils:162       [INFO]     Response: {"predictions": [[[0.923311, 1.1981051, 1.11143434, 2.45272899, 1.15551567] ..... 27186], [0.327044964, 0.49445653, 0.401043564,